<a href="https://colab.research.google.com/github/Santhose-DOC/Ex1-AAI/blob/main/experiment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas numpy pgmpy networkx matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 21.6 MB/s eta 0:00:00


Step 1: Import libraries

In [3]:
import pandas as pd
import numpy as np

from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

import networkx as nx
import matplotlib.pyplot as plt

Step 2: Load dataset

In [4]:
df = pd.read_csv("weatherAUSnew.csv")   # change path if needed


Step 3: Data preprocessing

In [5]:
# Remove rows where target is missing
df = df[pd.isnull(df['RainTomorrow']) == False]

# Drop Date column
df = df.drop(columns='Date')

# Fill missing numeric values with mean
numeric_columns = df.select_dtypes(include=['number']).columns
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())


Step 4: Discretization (Categorical variables)

In [6]:
df['WindGustSpeedCat'] = df['WindGustSpeed'].apply(
    lambda x: '0.<=40' if x <= 40 else
              '1.40-50' if 40 < x <= 50 else
              '2.>50'
)

df['Humidity9amCat'] = df['Humidity9am'].apply(
    lambda x: '1.>60' if x > 60 else '0.<=60'
)

df['Humidity3pmCat'] = df['Humidity3pm'].apply(
    lambda x: '1.>60' if x > 60 else '0.<=60'
)


Step 5: Prepare data for Bayesian Network

In [10]:
bn_df = df[
    ['WindGustSpeedCat', 'Humidity9amCat', 'Humidity3pmCat', 'RainTomorrow']
].copy()

# Convert all columns to categorical (string)
for col in bn_df.columns:
    bn_df[col] = bn_df[col].astype(str)

print(bn_df.head())

  WindGustSpeedCat Humidity9amCat Humidity3pmCat RainTomorrow
0          1.40-50          1.>60         0.<=60           No
1          1.40-50         0.<=60         0.<=60           No
2          1.40-50         0.<=60         0.<=60           No
3           0.<=40         0.<=60         0.<=60           No
4          1.40-50          1.>60         0.<=60           No


Step 6: Define Bayesian Network structure

In [13]:
model = DiscreteBayesianNetwork([
    ('Humidity9amCat', 'Humidity3pmCat'),
    ('Humidity3pmCat', 'RainTomorrow'),
    ('WindGustSpeedCat', 'RainTomorrow')
])


AttributeError: module 'networkx' has no attribute 'edge_dfs'